# Bin Dashboard — MQTT Live

This notebook subscribes to the simulator’s MQTT bin status topics and plots fill % over time.

## Prereqs
- Create `.env` with `HIVEMQ_USERNAME` and `HIVEMQ_PASSWORD` (or export them)
- Start the simulator in another terminal:

```bash
python -m simulated_city --steps 200
```

Install notebook dependencies:

```bash
python -m pip install -e ".[notebooks]"
```

In [1]:
from __future__ import annotations

import time

import altair as alt
import pandas as pd
from IPython.display import clear_output, display

from simulated_city.config import load_config
from simulated_city.dashboard_data import (
    drain_queue,
    event_from_payload,
    events_to_frame,
    start_mqtt_listener,
    stop_mqtt_listener,
)

In [ ]:
cfg = load_config()
TOPIC_FILTER = f"{cfg.mqtt.base_topic}/bins/+/+/status"

ALERT_THRESHOLD = 80
HISTORY_DAYS = 7  # Set None to show all data
REFRESH_S = 2.0
RUN_FOR_S = 60

print('Broker:', f"{cfg.mqtt.host}:{cfg.mqtt.port}", 'tls=', cfg.mqtt.tls)
print('Topic filter:', TOPIC_FILTER)

Broker: c451c402b7fb41b399936cd5727a1d3f.s1.eu.hivemq.cloud:8883 tls= True
Topic filter: simulated-city/bins/+/+/status


In [ ]:
# Starts a subscriber and renders a live chart for RUN_FOR_S seconds.
# Tip: stop early via Kernel -> Interrupt.

q, client = start_mqtt_listener(cfg.mqtt, TOPIC_FILTER)

# Problem: the MQTT broker keeps *retained* status messages, so if you start this
# notebook before the simulator, you'll initially receive the last values from a
# previous run.
#
# Fix: ignore everything until we observe a fresh `event="init"` message from the
# next simulator run.
WAIT_FOR_INIT_S = None  # Set to e.g. 30.0 to stop waiting and show retained state.
init_seen = False
init_deadline = time.time() + float(WAIT_FOR_INIT_S) if WAIT_FOR_INIT_S is not None else None

print("Connected. Waiting for a fresh run (init)...")

df = pd.DataFrame(
    {
        'ts': pd.Series(dtype='datetime64[ns, UTC]'),
        'series': pd.Series(dtype='string'),
        'fill_pct': pd.Series(dtype='int64'),
        'timestep_index': pd.Series(dtype='int64'),
        'event': pd.Series(dtype='string'),
    }
)

t_end = time.time() + float(RUN_FOR_S)
try:
    while time.time() < t_end:
        payloads = drain_queue(q)
        new_events = []
        for payload in payloads:
            try:
                new_events.append(event_from_payload(payload))
            except Exception:
                continue

        # Handle run boundaries.
        init_events = [e for e in new_events if e.event == 'init']
        if init_events:
            # New run started: clear any prior points (including retained old data).
            init_seen = True
            run_start_ts = max(e.ts for e in init_events)
            df = df[df['ts'] >= pd.Timestamp(run_start_ts)].copy() if not df.empty else df
            new_events = [e for e in new_events if e.ts >= run_start_ts]

        if not init_seen:
            if init_deadline is not None and time.time() >= init_deadline:
                # Fallback: if we missed init (e.g. started late), start plotting
                # whatever arrives (likely the current retained state).
                init_seen = True
            else:
                clear_output(wait=True)
                print('Waiting for the simulator to start a NEW run (init event)...')
                time.sleep(float(REFRESH_S))
                continue

        if new_events:
            df = pd.concat([df, events_to_frame(new_events)], ignore_index=True)
            df = df.sort_values(['ts', 'series'], kind='mergesort')
            df = df.drop_duplicates(subset=['ts', 'series', 'fill_pct', 'event'], keep='last').reset_index(drop=True)

        view = df
        if not view.empty:
            # If multiple runs are present (e.g. log replay or late init), keep
            # only the latest run based on the last init we have seen.
            init_ts = view.loc[view['event'] == 'init', 'ts'] if 'event' in view.columns else pd.Series(dtype='datetime64[ns, UTC]')
            if not init_ts.empty:
                view = view[view['ts'] >= init_ts.max()]

            diffs = view.sort_values(['series', 'ts']).groupby('series')['fill_pct'].diff()
            reset_rows = view.loc[diffs < 0, 'ts']
            if not reset_rows.empty:
                view = view[view['ts'] >= reset_rows.max()]

            if HISTORY_DAYS is not None and not view.empty:
                anchor = view['ts'].max()
                cutoff = anchor - pd.Timedelta(days=int(HISTORY_DAYS))
                view = view[view['ts'] >= cutoff]

        clear_output(wait=True)
        if view.empty:
            print('No status events yet for this run...')
        else:
            latest = view.sort_values('ts').groupby('series').tail(1)
            alerts = latest[latest['fill_pct'] >= int(ALERT_THRESHOLD)]
            if not alerts.empty:
                print('ALERT: Containers at or above threshold:')
                for row in alerts.itertuples(index=False):
                    print(f'  - {row.series} = {int(row.fill_pct)}%')
            else:
                print('All containers are below the alert threshold.')

            wide = view.pivot_table(index='ts', columns='series', values='fill_pct', aggfunc='last').sort_index()
            wide = wide.ffill().astype('float64')
            long = wide.reset_index().melt(id_vars='ts', var_name='series', value_name='fill_pct').dropna()

            chart = (
                alt.Chart(long)
                .mark_line(interpolate='step-after')
                .encode(
                    x=alt.X('ts:T', title=None),
                    y=alt.Y('fill_pct:Q', title='Fill %', scale=alt.Scale(domain=[0, 100])),
                    color=alt.Color('series:N', title=None),
                    tooltip=[
                        alt.Tooltip('ts:T', title='ts'),
                        alt.Tooltip('series:N', title='bin'),
                        alt.Tooltip('fill_pct:Q', title='fill_pct'),
                    ],
                )
            ).properties(height=350)

            display(chart)
            display(latest[['ts', 'series', 'fill_pct']].sort_values('series').reset_index(drop=True))

        time.sleep(float(REFRESH_S))
finally:
    stop_mqtt_listener(client)
    print('Disconnected.')

print('Live view finished.')


No status events yet for this run...
Disconnected.
Live view finished.
